In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2024-03-13 12:06:02.586023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 12:06:02.586095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 12:06:02.588862: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 12:06:02.608483: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 12:06:05.039116: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [2]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [3]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [4]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [5]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels

# Get list of image paths
image_paths = list(paths.list_images("../cell_images/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/27558
[INFO]: Processed 2000/27558
[INFO]: Processed 3000/27558
[INFO]: Processed 4000/27558
[INFO]: Processed 5000/27558
[INFO]: Processed 6000/27558
[INFO]: Processed 7000/27558
[INFO]: Processed 8000/27558
[INFO]: Processed 9000/27558
[INFO]: Processed 10000/27558
[INFO]: Processed 11000/27558
[INFO]: Processed 12000/27558
[INFO]: Processed 13000/27558
[INFO]: Processed 14000/27558
[INFO]: Processed 15000/27558
[INFO]: Processed 16000/27558
[INFO]: Processed 17000/27558
[INFO]: Processed 18000/27558
[INFO]: Processed 19000/27558
[INFO]: Processed 20000/27558
[INFO]: Processed 21000/27558
[INFO]: Processed 22000/27558
[INFO]: Processed 23000/27558
[INFO]: Processed 24000/27558
[INFO]: Processed 25000/27558
[INFO]: Processed 26000/27558
[INFO]: Processed 27000/27558
(27558, 224, 224, 3)
(27558,)


In [11]:
import gc
import os

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50" :
        labels = to_categorical(labels)

    else:
        data = data.reshape(data.shape[0],data.shape[1] * data.shape[2] * data.shape[3] )
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [9]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)


[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6672714078374455
Sensitivity: 0.6669085631349783
Specificity: 0.6676342525399129
AUC-ROC: 0.6672714078374457
MCC: 0.3345429037642499
Precision: 0.6673928830791576
F1 Score: 0.6671506352087114

[INFO] Fold 2 / 10 for LR
Accuracy: 0.6821480406386067
Sensitivity: 0.6821480406386067
Specificity: 0.6821480406386067
AUC-ROC: 0.6821480406386067
MCC: 0.36429608127721336
Precision: 0.6821480406386067
F1 Score: 0.6821480406386067

[INFO] Fold 3 / 10 for LR
Accuracy: 0.6868650217706821
Sensitivity: 0.716255442670537
Specificity: 0.6574746008708273
AUC-ROC: 0.6868650217706821
MCC: 0.3743773751760159
Precision: 0.6764907470870459
F1 Score: 0.6958054282692986

[INFO] Fold 4 / 10 for LR
Accuracy: 0.6727140783744557
Sensitivity: 0.690856313497823
Specificity: 0.6545718432510885
AUC-ROC: 0.6727140783744557
MCC: 0.34565577045174056
Precision: 0.6666666666666666
F1 Score: 0.6785459729151818

[INFO] Fold 5 / 10 for LR
Accuracy: 0.6730769230769231
Sensitivity: 0.7089985486211902
Specificity: 0.

In [10]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.6498548621190131
Sensitivity: 0.7670537010159652
Specificity: 0.532656023222061
AUC-ROC: 0.649854862119013
MCC: 0.3082986798307956
Precision: 0.6213991769547325
F1 Score: 0.6865865540759987

[INFO] Fold 2 / 10 for NB
Accuracy: 0.6444121915820029
Sensitivity: 0.7503628447024674
Specificity: 0.5384615384615384
AUC-ROC: 0.6444121915820029
MCC: 0.2955356821444854
Precision: 0.6191616766467066
F1 Score: 0.678477690288714

[INFO] Fold 3 / 10 for NB
Accuracy: 0.6436865021770682
Sensitivity: 0.7743105950653121
Specificity: 0.5130624092888244
AUC-ROC: 0.6436865021770682
MCC: 0.2977120483851791
Precision: 0.6139240506329114
F1 Score: 0.6848523748395379

[INFO] Fold 4 / 10 for NB
Accuracy: 0.625544267053701
Sensitivity: 0.7648766328011611
Specificity: 0.4862119013062409
AUC-ROC: 0.625544267053701
MCC: 0.2614447676088979
Precision: 0.5981838819523269
F1 Score: 0.6713375796178344

[INFO] Fold 5 / 10 for NB
Accuracy: 0.6157474600870827
Sensitivity: 0.7438316400

In [11]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.5947024673439768
Sensitivity: 0.8439767779390421
Specificity: 0.3454281567489115
AUC-ROC: 0.5947024673439768
MCC: 0.21849497612828753
Precision: 0.5631961259079903
F1 Score: 0.6755736276503049

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.5950653120464441
Sensitivity: 0.8403483309143687
Specificity: 0.3497822931785196
AUC-ROC: 0.5950653120464442
MCC: 0.21818883108326576
Precision: 0.563777994157741
F1 Score: 0.6748251748251749

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.5936139332365747
Sensitivity: 0.8592162554426706
Specificity: 0.32801161103047893
AUC-ROC: 0.5936139332365747
MCC: 0.22098470672355588
Precision: 0.5611374407582939
F1 Score: 0.6788990825688074

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.5925253991291727
Sensitivity: 0.8447024673439768
Specificity: 0.34034833091436867
AUC-ROC: 0.5925253991291728
MCC: 0.21430395079787284
Precision: 0.5615050651230101
F1 Score: 0.6745870762097943

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.589622641509434
Sensitivity

In [12]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.8203918722786647
Sensitivity: 0.795355587808418
Specificity: 0.8454281567489115
AUC-ROC: 0.8203918722786647
MCC: 0.6415885647131124
Precision: 0.8372803666921314
F1 Score: 0.815779679940454

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.8378084179970973
Sensitivity: 0.8127721335268505
Specificity: 0.862844702467344
AUC-ROC: 0.8378084179970973
MCC: 0.6764654062830212
Precision: 0.8556149732620321
F1 Score: 0.8336434685522888

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.8240203193033382
Sensitivity: 0.8062409288824384
Specificity: 0.841799709724238
AUC-ROC: 0.8240203193033382
MCC: 0.6484507275495354
Precision: 0.835966892400301
F1 Score: 0.8208348725526414

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.8174891146589259
Sensitivity: 0.7902757619738752
Specificity: 0.8447024673439768
AUC-ROC: 0.817489114658926
MCC: 0.6359208112610414
Precision: 0.8357636224098235
F1 Score: 0.8123834390152929

[INFO] Fold 5 / 10 for Random Forest


In [13]:
import lightgbm as lgb 
# LGBM
lgb_model = lgb.LGBMClassifier(random_state=0, n_jobs=-1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 91.785226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22033255
[LightGBM] [Info] Number of data points in the train set: 24802, number of used features: 150090
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.8465166908563135
Sensitivity: 0.8621190130624092
Specificity: 0.8309143686502177
AUC-ROC: 0.8465166908563135
MCC: 0.6933710419645234
Precision: 0.8360309641097818
F1 Score: 0.8488745980707395

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 90.756687 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22028712
[LightGBM] [Info] 

In [8]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the VGG16 model pre-trained on ImageNet data
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Build a new model using VGG16 as a feature extractor
with tf.device('/GPU:0'):
    model_vgg16 = Sequential()
    model_vgg16.add(vgg16_model)
    model_vgg16.add(Flatten())
    model_vgg16.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_vgg16.add(Dropout(0.5, seed = 0))
    model_vgg16.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_vgg16, 'VGG16', data, labels)

2024-03-13 12:07:10.023485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37960 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0



[INFO] Fold 1 / 10 for VGG16


2024-03-13 12:07:30.288148: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20


2024-03-13 12:07:45.147586: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-13 12:07:45.458244: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 12:07:48.928688: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa8871630c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 12:07:48.928745: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-03-13 12:07:48.944389: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710331669.253595  503337 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


388/388 [==============================] - 25s 46ms/step - loss: 0.7874 - accuracy: 0.9055
Epoch 2/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1934 - accuracy: 0.9308
Epoch 3/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1590 - accuracy: 0.9429
Epoch 4/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1401 - accuracy: 0.9491
Epoch 5/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1342 - accuracy: 0.9504
Epoch 6/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1287 - accuracy: 0.9513
Epoch 7/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1263 - accuracy: 0.9494
Epoch 8/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1112 - accuracy: 0.9554
Epoch 9/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1198 - accuracy: 0.9513
Epoch 10/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1161 - accurac

In [15]:

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the EfficientNetB7 model pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Build a new model using EfficientNetB7 as a feature extractor
with tf.device('/GPU:0'):
    model_efficientnet = Sequential()
    model_efficientnet.add(efficientnet_model)
    model_efficientnet.add(Flatten())
    model_efficientnet.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_efficientnet.add(Dropout(0.5, seed=0))
    model_efficientnet.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_efficientnet, 'EfficientNetB0', data, labels)



[INFO] Fold 1 / 10 for EfficientNetB0
Epoch 1/20
388/388 [==============================] - 34s 41ms/step - loss: 0.4029 - accuracy: 0.9163
Epoch 2/20
388/388 [==============================] - 17s 45ms/step - loss: 0.1826 - accuracy: 0.9352
Epoch 3/20
388/388 [==============================] - 18s 45ms/step - loss: 0.1576 - accuracy: 0.9431
Epoch 4/20
388/388 [==============================] - 16s 40ms/step - loss: 0.1487 - accuracy: 0.9439
Epoch 5/20
388/388 [==============================] - 15s 39ms/step - loss: 0.1467 - accuracy: 0.9454
Epoch 6/20
388/388 [==============================] - 15s 39ms/step - loss: 0.1420 - accuracy: 0.9487
Epoch 7/20
388/388 [==============================] - 15s 39ms/step - loss: 0.1276 - accuracy: 0.9511
Epoch 8/20
388/388 [==============================] - 15s 39ms/step - loss: 0.1261 - accuracy: 0.9527
Epoch 9/20
388/388 [==============================] - 15s 39ms/step - loss: 0.1223 - accuracy: 0.9540
Epoch 10/20
388/388 [======================

In [18]:
from tensorflow.keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the ResNet50 model pre-trained on ImageNet data
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build a new model using ResNet50 as a feature extractor
with tf.device('/GPU:0'):
    model_resnet50 = Sequential()
    model_resnet50.add(resnet50_model)
    model_resnet50.add(Flatten())
    model_resnet50.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_resnet50.add(Dropout(0.5, seed=0))
    model_resnet50.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_resnet50, 'ResNet50', data, labels)



[INFO] Fold 1 / 10 for ResNet50
Epoch 1/20
388/388 [==============================] - 28s 48ms/step - loss: 1.8163 - accuracy: 0.8305
Epoch 2/20
388/388 [==============================] - 18s 46ms/step - loss: 0.2853 - accuracy: 0.8874
Epoch 3/20
388/388 [==============================] - 18s 46ms/step - loss: 0.2451 - accuracy: 0.9036
Epoch 4/20
388/388 [==============================] - 19s 48ms/step - loss: 0.2268 - accuracy: 0.9100
Epoch 5/20
388/388 [==============================] - 18s 46ms/step - loss: 0.2127 - accuracy: 0.9141
Epoch 6/20
388/388 [==============================] - 18s 46ms/step - loss: 0.2038 - accuracy: 0.9162
Epoch 7/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1870 - accuracy: 0.9232
Epoch 8/20
388/388 [==============================] - 18s 46ms/step - loss: 0.1880 - accuracy: 0.9212
Epoch 9/20
388/388 [==============================] - 18s 47ms/step - loss: 0.1775 - accuracy: 0.9267
Epoch 10/20
388/388 [============================

In [19]:
# Save results to a CSV file
results_df.to_csv('DS1_baseline.csv', index=False)

In [20]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.677190,0.692067,0.662312,0.677189,0.354653,0.672212,0.681887,3978.546875,14173.192798
1,NB,0.635024,0.755716,0.514334,0.635025,0.278357,0.608822,0.674304,3956.449219,1237.740232
2,KNN,0.595072,0.847376,0.342769,0.595072,0.220247,0.563207,0.676654,3974.527344,2036.960241
3,Random Forest,0.824370,0.796211,0.852528,0.824370,0.649857,0.843834,0.819272,4082.570312,1909.773281
4,LGBM,0.852275,0.870890,0.833659,0.852275,0.705089,0.839725,0.854994,65518.851562,18971.517619
5,VGG16,0.983526,0.990493,0.976559,0.983526,0.967209,0.977055,0.983695,10946.582031,2737.650421
6,EfficientNetB0,0.980914,0.991872,0.969954,0.980913,0.962149,0.970761,0.981157,11667.882812,2297.048106
7,ResNet50,0.982873,0.977430,0.988316,0.982873,0.965862,0.988165,0.982736,11611.468750,2671.979346


In [13]:
results_df = pd.read_csv('DS1_baseline.csv')